In [38]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field, ConfigDict
from typing import List, Optional
from datetime import datetime
import os

client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

class PointTimeRequest(BaseModel):
    # lat: float = Field(..., ge=-90, le=90)
    # lon: float = Field(..., ge=-180, le=360)
    location: str
    variables: List[str]
    from_datetime: str = Field(..., title='Start time',
                               description='Start time for the data. formatted as :%Y-%m-%dT00:00:00Z')
    interval: Optional[str] = Field(None, title='Time interval', description='Time interval for the data. For example, "1h" for hourly data, "3h" for 3 hour intervals etc.')
    repeat: Optional[int] = Field(None, title='Number of intervals', description='Number of intervals to repeat the data for. For example, if interval is "1h", "3" will request 3 hours of data.')

    model_config = ConfigDict(arbritary_types_allowed=True)

metservice_variables = [
'air.humidity.at-2m',
'air.pressure.at-sea-level',
'air.temperature.at-2m',
'air.visibility',
'atmosphere.convective.potential.energy',
'cloud.base.height',
'cloud.cover',
'precipitation.rate',
'radiation.flux.downward.longwave',
'radiation.flux.downward.shortwave',
'wind.direction.at-10m',
'wind.direction.at-100m',
'wind.speed.at-10m',
'wind.speed.at-100m',
'wind.speed.eastward.at-100m',
'wind.speed.eastward.at-10m',
'wind.speed.gust.at-10m',
'wind.speed.northward.at-100m',
'wind.speed.northward.at-10m',
'wave.height',
'wave.height.max',
'wave.direction.peak',
'wave.period.peak',
'wave.height.above-8s',
'wave.height.below-8s',
'wave.period.above-8s.peak',
'wave.period.below-8s.peak',
'wave.direction.above-8s.peak',
'wave.direction.below-8s.peak',
'wave.direction.mean',
'wave.directional-spread',
'wave.period.tm01.mean',
'wave.period.tm02.mean',
'current.speed.eastward.at-sea-surface',
'current.speed.eastward.at-sea-surface-no-tide',
'current.speed.eastward.barotropic',
'current.speed.eastward.barotropic-no-tide',
'current.speed.northward.at-sea-surface',
'current.speed.northward.at-sea-surface-no-tide',
'current.speed.northward.barotropic',
'current.speed.northward.barotropic-no-tide',
'sea.temperature.at-surface',
'sea.temperature.at-surface-anomaly',
]

In [46]:
user_message = 'what will the weather be like in Auckland tomorrow?'

response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    response_model=PointTimeRequest,
    messages=[
        {"role": "user", "content":
        f"""Based on the user message below, complete the JSON structure, which will form the basis of the request to a weather API. Underneath the user message is a list of variables that can be requested from the API, please only select the most relevant variables to answer the query. In case the date is relational, today's date is {datetime.now().isoformat()}. If the request doesn't specify a time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
        ______________________________
        USER MESSAGE
        ______________________________
        {user_message}
        ______________________________
        VARIABLES
        ______________________________
        {metservice_variables}"""
        }
    ],
    
)

assert isinstance(response, PointTimeRequest)
print(response)

location='Auckland' variables=['air.humidity.at-2m', 'air.pressure.at-sea-level', 'air.temperature.at-2m', 'air.visibility', 'cloud.cover', 'precipitation.rate', 'wind.direction.at-10m', 'wind.speed.at-10m'] from_datetime='2024-02-08T00:00:00Z' interval='1h' repeat=24


In [43]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="Geopy Library")
location = loc.geocode(response.location)
lon = location.longitude
lat = location.latitude



In [47]:
from requests import post


resp = post(
    "https://forecast-v2.metoceanapi.com/point/time",
    headers={"x-api-key": os.environ["METSERVICE_KEY"]},
    json={
        "points": [{
            "lon": lon,
            "lat": lat
        }],
        "variables": response.variables
        ,
        "time": {
            "from": response.from_datetime,
            "interval": response.interval,
            "repeat": response.repeat
        }
    }
)

In [48]:
import json

if resp.status_code != 200:
    raise ValueError("{}: {}", resp.status_code, resp.text)

print(json.dumps(resp.json(), indent=1))

{
 "dimensions": {
  "point": {
   "type": "point",
   "units": "unknown",
   "data": [
    {
     "lon": 174.7631803,
     "lat": -36.852095
    }
   ]
  },
  "time": {
   "type": "time",
   "units": "unknown",
   "data": [
    "2024-02-08T00:00:00Z",
    "2024-02-08T01:00:00Z",
    "2024-02-08T02:00:00Z",
    "2024-02-08T03:00:00Z",
    "2024-02-08T04:00:00Z",
    "2024-02-08T05:00:00Z",
    "2024-02-08T06:00:00Z",
    "2024-02-08T07:00:00Z",
    "2024-02-08T08:00:00Z",
    "2024-02-08T09:00:00Z",
    "2024-02-08T10:00:00Z",
    "2024-02-08T11:00:00Z",
    "2024-02-08T12:00:00Z",
    "2024-02-08T13:00:00Z",
    "2024-02-08T14:00:00Z",
    "2024-02-08T15:00:00Z",
    "2024-02-08T16:00:00Z",
    "2024-02-08T17:00:00Z",
    "2024-02-08T18:00:00Z",
    "2024-02-08T19:00:00Z",
    "2024-02-08T20:00:00Z",
    "2024-02-08T21:00:00Z",
    "2024-02-08T22:00:00Z",
    "2024-02-08T23:00:00Z",
    "2024-02-09T00:00:00Z"
   ]
  }
 },
 "noDataReasons": {
  "ERROR_INTERNAL": 5,
  "FILL": 2,
  "GAP"

In [50]:
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "user", "content":
         f"""Based on the weather data provided below, answer the user query. Make sure to use the data to answer the query as accurately as possible. You must pretend like you are replying directly to the user and you have just have the benefit of the weather data to answer the query. Answer in natural language and just give the topline details that someone would want when asking about the weather.
        ______________________________
        USER MESSAGE
        ______________________________
        {user_message}
        ______________________________
        WEATHER DATA
        ______________________________
        {json.dumps(resp.json(), indent=1)}"""
         }
    ],

)

print(response)

ChatCompletion(id='chatcmpl-8pYaERK05DjAziKf2Qu4X5L1sAWcX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in Auckland tomorrow will show a range of changing conditions throughout the day. Humidity levels will start at around 60% in the early hours and will progressively increase, reaching up to around 84% by noon and peaking at about 91% in the late afternoon before dropping slightly in the evening.\n\nTemperatures will begin at around 24°C (297K) in the early morning, then gradually decrease through the day, reaching a low of approximately 17°C (290K) in the late evening. Towards the end of the day, temperatures will rise again, finishing at about 23°C (296K).\n\nVisibility is expected to decrease throughout the day, starting at over 42 km early in the morning and dropping to about 20 km by the evening, indicating possibly hazier conditions as the day progresses.\n\nCloud cover will be minimal in the early morning but 